# Chapter 3: Mapping sea ice concentration

Creators: Ann Windnagel and Robyn Marowitz

Affiliation: [National Snow And Ice Data Center](https://nsidc.org/home)

In [2]:
from datetime import date, timedelta
from netCDF4 import Dataset

import xarray as xr
import numpy
import pandas
import packaging
import aiohttp

import fsspec
import pystac
import netCDF4
from netCDF4 import Dataset
import h5netcdf
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

Matplotlib is building the font cache; this may take a moment.


## Below we will repeat some code from Notebook 2 to access the STAC Item and the NetCDF 

In [9]:
collection = pystac.Collection.from_file('https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/collection.json')
daily_sh_2023_item = collection.get_item("seaice_conc_daily_sh_2023_v04r00", recursive=True)
netcdf_url = daily_sh_2023_item.assets['netcdf'].href
fs = fsspec.filesystem('https')
ds = xr.open_dataset(fs.open(netcdf_url))

In [10]:
# Pull out sea ice concentration variable
cdr_seaice_conc = ds.variables['cdr_seaice_conc']
cdr_seaice_conc

<xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:            NOAA/NSIDC Climate Data Record of Passive Microwave...
    standard_name:        sea_ice_area_fraction
    units:                1
    flag_values:          [251 252 253 254 255]
    flag_meanings:        pole_hole lakes coastal land_mask missing_data
    datum:                +ellps=urn:ogc:def:crs:EPSG::4326
    grid_mapping:         projection
    reference:            https://nsidc.org/data/g02202/versions/4/
    ancillary_variables:  stdev_of_cdr_seaice_conc qa_of_cdr_seaice_conc
    valid_range:          [  0 100]
    cell_methods:         tdim: mean

## Exploring the Sea Ice Concentratino Vairable
Above we have pulled out the Sea Ice Concentration Variable from the NetCDF and now we will explore the data and flag values.

In [22]:
test = cdr_seaice_conc.where(cdr_seaice_conc <= 1)
test.max()

<xarray.Variable ()> Size: 4B
array(1., dtype=float32)

In [23]:
# Filter out any concentrations less than 15%
#sic_15 = ma.masked_outside(seaice_conc_np_2d_percent, 15, 255)
#filter out the ocean pixels = 0
ocean = ma.masked_outside(seaice_conc_np_2d_percent, 0, 0)

# Only keep sea ice conentration i.e. Remove the land (254)
sic_15 = ma.masked_outside(seaice_conc_np_2d_percent, 15, 100)

# filter out the land pixels = 254
land = ma.masked_outside(seaice_conc_np_2d_percent, 254, 254)
coast = ma.masked_outside(seaice_conc_np_2d_percent, 253, 253)
lakes = ma.masked_outside(seaice_conc_np_2d_percent, 252, 252)

fig, ax1 = plt.subplots(figsize=(22, 16))
seaice_15_plot = ax1.imshow(sic_15, cmap='Blues_r', interpolation='none')
land_plot = ax1.imshow(land, cmap='Accent_r', interpolation='none')
coast_plot = ax1.imshow(coast, cmap='Accent_r', interpolation='none')
ocean_plot = ax1.imshow(ocean, cmap='PuBu_r', interpolation='none')
lake_plot = ax1.imshow(lakes, cmap='PuBu_r', interpolation='none')
plt.axis('off')

# Save to tif image
hemi = filename[18:20]
tif_file_name = 'CDRV4-sic-15percent-edge-' + hemi + '-' + str(human_date) + '.tif'
plt.savefig(tif_file_name,bbox_inches='tight')
plt.show()

NameError: name 'ma' is not defined